In [1]:
import gensim
from gensim.models import Doc2Vec, KeyedVectors
import logging
import time
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import requests
import matplotlib.pyplot as plt
import gensim.downloader as api # Downloading the Google pretrained Word2Vec Model

In [2]:
import numpy as np
import requests
import random
from tqdm.notebook import tqdm
#tqdm.pandas()

In [3]:
import os
import json
import glob
import re
import sys
import collections
from nltk import flatten
import dask
from dask import delayed,compute
import dask.dataframe as dd
from dask.multiprocessing import get
import logging
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
avg_word_vectors = pd.read_csv('book_embeddings_from_avg_word_vectors.csv')

In [9]:
paragraph_vectors = pd.read_csv('book_embeddings_from_paragraph_vectors.csv')

In [6]:
bert_vectors = pd.read_csv('book_embeddings_from_bert.csv')

In [7]:
books_df = pd.read_csv('unique_books.csv')

In [76]:
books_df.loc[45:65,:]

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
45,703600,1,47,3,4101,369,19063.0,375831002,9.780376e+12,The Book Thief,4.36,1159741.0,"['Markus Zusak', 'Trudy White']",HERE IS A SMALL FACT:YOU ARE GOING TO DIE.1939...,9.780550e+12,The Book Thief,"['Historical', 'Historical Fiction', 'Fiction'...",here is a small fact you...
46,755570,4,1296,3,7232,148,15860.0,61015733,9.780061e+12,State of Fear,3.69,65397.0,['Michael Crichton'],"In Paris, a physicist dies after performing a ...",9.780060e+12,State of Fear,"['Fiction', 'Thriller', 'Science Fiction']",in paris a physicist dies ...
47,756687,4,86,4,7232,282,32542.0,385338600,9.780385e+12,A Time to Kill,4.03,597775.0,['John Grisham'],Before The Firm and The Pelican Brief made him...,9.780390e+12,A Time to Kill,"['Fiction', 'Mystery', 'Thriller', 'Mystery', ...",before the firm and the pelic...
48,762531,4,123,4,7232,482,5358.0,582418275,9.780582e+12,The Firm,3.99,488269.0,"['Robin A.H. Waterfield', 'John Grisham']","Mitch McDeere is young, intelligent and ambiti...",9.780580e+12,The Firm,"['Fiction', 'Thriller', 'Mystery', 'Mystery', ...",mitch mcdeere is young inte...
49,768834,4,42,5,7232,215,1934.0,451529308,9.780452e+12,Little Women,4.04,1257121.0,['Louisa May Alcott'],"Generations of readers young and old, male and...",9.780450e+12,Little Women,"['Classics', 'Fiction', 'Historical', 'Histori...",generations of readers young and...
50,875324,4,249,4,7232,66,4588.0,618711651,9.780619e+12,Extremely Loud and Incredibly Close,3.97,294726.0,['Jonathan Safran Foer'],Jonathan Safran Foer emerged as one of the mos...,9.780620e+12,Extremely Loud and Incredibly Close,"['Fiction', 'Contemporary', 'Historical', 'His...",jonathan safran foer emerged as ...
51,878843,4,158,4,7232,381,6310.0,142403881,9.780142e+12,Charlie and the Chocolate Factory,4.10,453959.0,"['Roald Dahl', 'Quentin Blake']",Willy Wonka's famous chocolate factory is open...,9.780140e+12,Charlie and the Chocolate Factory,"['Childrens', 'Fantasy', 'Fiction', 'Classics']",willy wonka s famous chocolate ...
52,890911,4,373,4,7232,478,6319.0,141311371,9.780141e+12,The BFG,4.22,245855.0,"['Roald Dahl', 'Quentin Blake']",Captured by a giant! The BFG is no ordinary bo...,9.780140e+12,The BFG,"['Childrens', 'Fantasy', 'Fiction']",captured by a giant the ...
53,896689,4,184,4,7232,498,39988.0,141301066,9.780141e+12,Matilda,4.29,440743.0,"['Roald Dahl', 'Quentin Blake']",Matilda is a little girl who is far too good t...,9.780140e+12,Matilda,"['Childrens', 'Fiction', 'Fantasy', 'Classics']",matilda is a little girl who ...
54,906309,4,495,4,7232,473,4953.0,375725784,9.780376e+12,A Heartbreaking Work of Staggering Genius,3.68,145459.0,['Dave Eggers'],'When you read his extraordinary memoir you do...,9.780380e+12,A Heartbreaking Work of Staggering Genius,['Fiction'],when you read his extraordinary...


In [42]:
books_df.tail()

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
4322,6862729,30013,7467,4,24827,294,28016509.0,425285049,9.780425e+12,The Girl Before,3.70,23500.0,['J.P. Delaney'],"In the tradition of The Girl on the Train, The...",9.780430e+12,The Girl Before,"['Thriller', 'Mystery', 'Fiction', 'Suspense',...",in the tradition of the girl ...
4323,6862861,6120,6443,3,43815,182,30008702.0,385542070,9.780386e+12,Nutshell,3.71,10904.0,['Ian McEwan'],Nutshell is a classic story of murder and dece...,9.780390e+12,Nutshell,"['Fiction', 'Contemporary', 'Mystery', 'Litera...",nutshell is a classic story o...
4324,6863071,31288,6428,5,33660,87,31538635.0,1781106282,9.781781e+12,"Short Stories from Hogwarts of Heroism, Hardsh...",4.22,16285.0,"['J.K. Rowling', 'MinaLima']",‘Minerva was the Roman goddess of warriors and...,9.781780e+12,"Short Stories from Hogwarts of Heroism, Hardsh...","['Fantasy', 'Short Stories', 'Fiction', 'Young...",minerva was the roman goddes...
4325,6863235,12503,8685,4,3215,37,28965131.0,125011117X,9.781250e+12,Behind Her Eyes,3.77,17944.0,['Sarah Pinborough'],Why is everyone talking about the ending of Sa...,9.781250e+12,Behind Her Eyes,"['Thriller', 'Mystery', 'Fiction', 'Suspense',...",why is everyone talking about ...
4326,6863345,53098,9569,5,35429,250,32075671.0,62498533,9.780062e+12,The Hate U Give,4.62,32610.0,"['Angie Thomas', 'Angie Thomas']",Sixteen-year-old Starr Carter moves between tw...,9.780060e+12,The Hate U Give,"['Young Adult', 'Contemporary', 'Fiction', 'Re...",sixteen year old starr carter moves...


In [22]:
idx = books_df[books_df['original_title']=='The Da Vinci Code'].index.item()

In [52]:
books_df[books_df['book_title']=='The Da Vinci Code'].index.item()

1

In [18]:
paragraph_vectors.loc[0,:]

Unnamed: 0    0.000000
0            -0.000267
1             0.000167
2             0.000014
3             0.000536
                ...   
763           0.000175
764          -0.000417
765          -0.000238
766          -0.000499
767          -0.000490
Name: 0, Length: 769, dtype: float64

In [21]:
cosine_similarities.shape

(4327, 4327)

In [44]:
def recommend(embeddings, idx):
    cosine_similarities = cosine_similarity(embeddings, embeddings)
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:10]
    book_indices = [i[0] for i in sim_scores]
    recommend = books_df.iloc[book_indices]
    return recommend

## Paragraph vectors

In [57]:
def recommend_using_doc2vec(name_of_book):
    if name_of_book not in list(books_df['book_title']):
        print('Book Chosen not available')
        return
    else:
        idx = books_df[books_df['original_title']==name_of_book].index.item()
        return recommend(paragraph_vectors, idx)

In [58]:
recommend_using_doc2vec('The Da Vinci Code')

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
19,309486,4,5,4,7232,498,4671.0,743273567,9.780743e+12,The Great Gatsby,3.89,2683664.0,['F. Scott Fitzgerald'],These sumptuous new hardback editions mark the...,9.780140e+12,The Great Gatsby,"['Classics', 'Fiction', 'Academic', 'School', ...",these sumptuous new hardback edi...
37,595293,4,222,4,7232,269,7869.0,752864327,9.780753e+12,The Bourne Identity,3.98,327122.0,['Robert Ludlum'],Alternate cover for this ISBN can be found her...,9.780750e+12,The Bourne Identity,"['Fiction', 'Thriller', 'Mystery', 'Suspense',...",alternate cover for this isbn ...
62,968617,9,101,5,34632,478,4137.0,349113912,9.780349e+12,Me Talk Pretty One Day,3.97,495736.0,['David Sedaris'],David Sedaris' move to Paris from New York ins...,9.780350e+12,Me Talk Pretty One Day,"['Nonfiction', 'Humor', 'Autobiography', 'Memo...",david sedaris move to paris...
31,545870,8,493,5,38480,492,11138.0,684823780,9.780685e+12,Mere Christianity,4.31,184759.0,['C.S. Lewis'],"In the classic Mere Christianity, C.S. Lewis, ...",9.780680e+12,Mere Christianity,"['Nonfiction', 'Christian', 'Religion', 'Relig...",in the classic mere christianity...
61,966440,9,658,4,34632,444,3805.0,1841156736,9.781841e+12,The Corrections,3.78,117657.0,['Jonathan Franzen'],Winner of the National Book Award for FictionN...,9.781840e+12,The Corrections,"['Fiction', 'Contemporary', 'Novels']",winner of the national book a...
46,755570,4,1296,3,7232,148,15860.0,61015733,9.780061e+12,State of Fear,3.69,65397.0,['Michael Crichton'],"In Paris, a physicist dies after performing a ...",9.780060e+12,State of Fear,"['Fiction', 'Thriller', 'Science Fiction']",in paris a physicist dies ...
32,547685,8,362,5,38480,148,11149.0,60652896,9.780061e+12,A Grief Observed,4.21,116277.0,['C.S. Lewis'],"Written with love, humility, and faith, this b...",9.780060e+12,A Grief Observed,"['Nonfiction', 'Christian', 'Religion', 'Relig...",written with love humility ...
43,701342,1,1796,5,4101,414,68210.0,031242440X,9.780312e+12,Gilead,3.84,54081.0,['Marilynne Robinson'],"Twenty-four years after her first novel, House...",9.780310e+12,Gilead,"['Fiction', 'Historical', 'Historical Fiction'...",twenty four years after her firs...
54,906309,4,495,4,7232,473,4953.0,375725784,9.780376e+12,A Heartbreaking Work of Staggering Genius,3.68,145459.0,['Dave Eggers'],'When you read his extraordinary memoir you do...,9.780380e+12,A Heartbreaking Work of Staggering Genius,['Fiction'],when you read his extraordinary...


In [64]:
recommend_using_doc2vec('Harry Potter and the Half-Blood Prince')

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
171,1620196,28,3504,4,48890,158,67.0,61159174,9.780061e+12,The Known World,3.82,26550.0,['Edward P. Jones'],One of the most acclaimed novels in recent mem...,9.780060e+12,The Known World,"['Historical', 'Historical Fiction', 'Fiction'...",one of the most acclaimed nov...
145,1518910,24,2188,4,47129,308,14250.0,571171788,9.780571e+12,Pigs in Heaven,3.97,43374.0,['Barbara Kingsolver'],"Mother and adopted daughter, Taylor and Turtle...",9.780570e+12,Pigs in Heaven,"['Fiction', 'Contemporary', 'Adult Fiction', '...",mother and adopted daughter ...
159,1573085,22,1498,4,48089,482,11602.0,340818670,9.780341e+12,Hearts in Atlantis,3.80,65063.0,['Stephen King'],"Five interconnected, sequential narratives, se...",9.780340e+12,Hearts in Atlantis,"['Horror', 'Fiction', 'Fantasy']",five interconnected sequential ...
95,1213767,11,233,5,27151,371,9712.0,140003468X,9.781400e+12,El amor en los tiempos del cólera,3.89,283806.0,['Gabriel García Márquez'],"In their youth, Florentino Ariza and Fermina D...",9.780310e+12,El amor en los tiempos del cólera,"['Fiction', 'Classics', 'Historical', 'Histori...",in their youth florentino a...
188,1700057,32,416,5,47500,288,6327.0,590032496,9.780590e+12,The Witches,4.17,221431.0,"['Roald Dahl', 'Quentin Blake']",This is not a fairy-tale. This is about real w...,9.780590e+12,The Witches,"['Childrens', 'Fantasy', 'Fiction']",this is not a fairy tale ...
318,2304980,73,618,4,51338,234,8921.0,451528018,9.780452e+12,The Hound of the Baskervilles,4.09,154517.0,"['Arthur Conan Doyle', 'Anne Perry', 'Maria Bu...","'Mr. Holmes, they were the footprints of a gig...",9.780450e+12,The Hound of the Baskervilles,"['Classics', 'Mystery', 'Fiction', 'Mystery', ...",mr holmes they were t...
134,1499089,4,3469,5,7232,491,38169.0,60741872,9.780061e+12,"Alas, Babylon",4.07,28386.0,['Pat Frank'],"""Alas, Babylon."" Those fateful words heralded ...",9.780060e+12,"Alas, Babylon","['Fiction', 'Science Fiction', 'Apocalyptic', ...",`` alas babylon tho...
154,1535832,22,1142,4,48089,12,816.0,60512806,9.780061e+12,Cryptonomicon,4.25,72683.0,['Neal Stephenson'],"Cryptonomicon zooms all over the world, careen...",9.780060e+12,Cryptonomicon,"['Fiction', 'Science Fiction', 'Historical', '...",cryptonomicon zooms all over the...
371,2487734,94,1079,4,22103,198,13103.0,446671002,9.780447e+12,The Celestine Prophecy,3.60,72675.0,"['James Redfield', 'James Redfield']",The Celestine Prophecy contains secrets that a...,9.780450e+12,The Celestine Prophecy,"['Fiction', 'Spirituality', 'Philosophy', 'Sel...",the celestine prophecy contains ...


In [78]:
recommend_using_doc2vec("Barrel Fever: Stories and Essays")

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
714,3388942,171,5399,5,31591,26,464260.0,067974472X,9.780680e+12,The Fire Next Time,4.45,19816.0,['James Baldwin'],A national bestseller when it first appeared i...,9.780680e+12,The Fire Next Time,"['Nonfiction', 'Writing', 'Essays', 'Race', 'C...",a national bestseller when it ...
823,3550867,233,2108,3,8685,46,395233.0,140446745,9.780140e+12,The Letters of Vincent van Gogh,3.99,48040.0,"['Vincent van Gogh', 'Ronald de Leeuw', 'Arnol...",A new selection of post-impressionist painter ...,9.780140e+12,The Letters of Vincent van Gogh,"['Art', 'Nonfiction', 'Biography', 'History']",a new selection of post impressi...
1173,3945364,384,5263,5,2830,292,97898.0,1400031818,9.781400e+12,The Full Cupboard of Life,4.02,23089.0,['Alexander McCall Smith'],"Precious Ramotswe, of the No.1 Ladies Detectiv...",9.781400e+12,The Full Cupboard of Life,"['Mystery', 'Fiction', 'Cultural', 'Africa', '...",precious ramotswe of the no...
1818,4436458,875,2731,5,46953,196,1062516.0,590930028,9.780591e+12,"No, David!",4.11,36963.0,['David Shannon'],When author and artist David Shannon was five ...,9.780590e+12,"No, David!","['Childrens', 'Picture Books', 'Childrens', 'R...",when author and artist david ...
837,3567123,229,3966,3,7093,459,4909.0,743243781,9.780743e+12,Teacher Man,3.73,23736.0,['Frank McCourt'],McCourt's long-awaited book about how his thir...,9.780740e+12,Teacher Man,"['Nonfiction', 'Autobiography', 'Memoir', 'Bio...",mccourt s long awaited book abo...
507,2747968,137,1178,3,25558,182,62446.0,451528352,9.780452e+12,The Prince and the Pauper,3.84,84682.0,"['Mark Twain', 'Everett H. Emerson']","This treasured historical satire, played out i...",9.780450e+12,The Prince and the Pauper,"['Classics', 'Fiction', 'Historical', 'Histori...",this treasured historical satire ...
1327,4061550,404,1598,3,27238,159,29999.0,752865331,9.780753e+12,The Maltese Falcon,3.92,58742.0,['Dashiell Hammett'],Sam Spade is hired by the fragrant Miss Wonder...,9.780750e+12,The Maltese Falcon,"['Mystery', 'Fiction', 'Classics', 'Mystery', ...",sam spade is hired by the ...
1423,4180290,473,2735,3,45472,113,16429.0,553804790,9.780554e+12,The Husband,3.80,36233.0,['Dean Koontz'],"With each and every new novel, Dean Koontz rai...",9.780550e+12,The Husband,"['Thriller', 'Fiction', 'Horror', 'Mystery', '...",with each and every new novel...
1209,3975060,387,1279,4,47776,314,13083.0,440421705,9.780440e+12,Hoot,3.81,75275.0,['Carl Hiaasen'],"Unfortunately, Roy's first acquaintance in Flo...",9.780440e+12,Hoot,"['Young Adult', 'Fiction', 'Realistic Fiction'...",unfortunately roy s first ...


## Average of word vectors

In [59]:
def recommend_using_word2vec(name_of_book):
    if name_of_book not in list(books_df['book_title']):
        print('Book Chosen not available')
        return
    else:
        idx = books_df[books_df['original_title']==name_of_book].index.item()
        return recommend(avg_word_vectors, idx)

In [62]:
recommend_using_word2vec('The Da Vinci Code')

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
4,49199,2,301,5,41929,194,4900.0,1892295490,9.781892e+12,Heart of Darkness,3.42,255576.0,['Joseph Conrad'],Dark allegory describes the narrator's journey...,9.781890e+12,Heart of Darkness,"['Classics', 'Fiction', 'Literature', 'Cultura...",dark allegory describes the narr...
5,72564,2,8519,5,41929,369,401876.0,385337388,9.780385e+12,The House of God,3.95,8812.0,"['Samuel Shem', 'John Updike']",The hilarious novel of the healing arts that r...,9.780390e+12,The House of God,"['Health', 'Medicine', 'Fiction', 'Medical', '...",the hilarious novel of the he...
3,15287,2,33,4,41929,98,930.0,739326228,9.780739e+12,Memoirs of a Geisha,4.08,1300209.0,['Arthur Golden'],"A literary sensation and runaway bestseller, t...",9.780740e+12,Memoirs of a Geisha,"['Fiction', 'Historical', 'Historical Fiction'...",a literary sensation and runaway...
2,13461,2,315,3,41929,193,4894.0,91883768,9.780092e+12,Who Moved My Cheese?,3.74,237272.0,"['Spencer Johnson', 'Kenneth H. Blanchard']","Written by Spencer Johnson, coauthor of The On...",9.780090e+12,Who Moved My Cheese?,"['Business', 'Self Help', 'Nonfiction', 'Psych...",written by spencer johnson ...
6,72687,4,264,3,7232,432,3876.0,743297334,9.780743e+12,The Sun Also Rises,3.83,284988.0,['Ernest Hemingway'],"Paris in the twenties: Pernod, parties and exp...",9.780680e+12,The Sun Also Rises,"['Classics', 'Fiction', 'Literature', 'Novels']",paris in the twenties perno...
10,110176,4,21,5,7232,12,2.0,439358078,9.780439e+12,Harry Potter and the Order of the Phoenix,4.46,1735368.0,"['J.K. Rowling', 'Mary GrandPré']",There is a door at the end of a silent corrido...,9.780440e+12,Harry Potter and the Order of the Phoenix,"['Fantasy', 'Young Adult', 'Fiction']",there is a door at the end...
8,79240,4,18,5,7232,425,5.0,043965548X,9.780440e+12,Harry Potter and the Prisoner of Azkaban,4.53,1832823.0,"['J.K. Rowling', 'Mary GrandPré']",Harry Potter's third year at Hogwarts is full ...,9.780440e+12,Harry Potter and the Prisoner of Azkaban,"['Fantasy', 'Young Adult', 'Fiction']",harry potter s third year at...
7,76848,4,388,4,7232,491,50.0,689840926,9.780690e+12,Hatchet,3.68,222553.0,['Gary Paulsen'],Brian is on his way to Canada to visit his est...,9.780690e+12,Hatchet,"['Young Adult', 'Fiction', 'Adventure', 'Class...",brian is on his way to can...
9,95095,4,27,5,7232,354,1.0,439785960,9.780440e+12,Harry Potter and the Half-Blood Prince,4.54,1678823.0,"['J.K. Rowling', 'Mary GrandPré']",When Harry Potter and the Half-Blood Prince op...,9.780440e+12,Harry Potter and the Half-Blood Prince,"['Fantasy', 'Young Adult', 'Fiction']",when harry potter and the hal...


In [63]:
recommend_using_word2vec('Harry Potter and the Half-Blood Prince')

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
13,281870,4,255,2,7232,388,662.0,452011876,9.780452e+12,Atlas Shrugged,3.68,283295.0,"['Ayn Rand', 'Leonard Peikoff']",This is the story of a man who said that he wo...,9.780450e+12,Atlas Shrugged,"['Fiction', 'Classics', 'Philosophy', 'Literat...",this is the story of a man...
23,458127,4,113,4,7232,24,168668.0,684833395,9.780685e+12,Catch-22,3.98,563265.0,['Joseph Heller'],"The novel is set during World War II, from 194...",9.780680e+12,Catch-22,"['Classics', 'Fiction', 'War', 'Historical', '...",the novel is set during world...
20,392506,4,8,4,7232,139,5107.0,316769177,9.780317e+12,The Catcher in the Rye,3.79,2044241.0,['J.D. Salinger'],The hero-narrator of The Catcher in the Rye is...,9.780320e+12,The Catcher in the Rye,"['Classics', 'Fiction', 'Young Adult', 'Litera...",the hero narrator of the catcher...
17,301904,4,660,3,7232,479,7672.0,60541830,9.780061e+12,Congo,3.55,129811.0,['Michael Crichton'],"Deep in the African rain forest, near the lege...",9.780060e+12,Congo,"['Fiction', 'Science Fiction', 'Thriller', 'Ad...",deep in the african rain fore...
27,483147,8,2584,5,38480,232,393199.0,015626224X,9.780156e+12,Down and Out in Paris and London,4.10,34739.0,['George Orwell'],This unusual fictional account - in good part ...,9.780160e+12,Down and Out in Paris and London,"['Nonfiction', 'Classics', 'Biography', 'Autob...",this unusual fictional account ...
10,110176,4,21,5,7232,12,2.0,439358078,9.780439e+12,Harry Potter and the Order of the Phoenix,4.46,1735368.0,"['J.K. Rowling', 'Mary GrandPré']",There is a door at the end of a silent corrido...,9.780440e+12,Harry Potter and the Order of the Phoenix,"['Fantasy', 'Young Adult', 'Fiction']",there is a door at the end...
29,541721,8,778,5,38480,33,30597.0,451527887,9.780452e+12,Notre-Dame de Paris,3.97,119851.0,"['Alain Erlande-Brandenburg', 'Caroline Rose',...",The cathedral of Notre-Dame de Paris is one of...,9.780810e+12,Notre-Dame de Paris,"['Classics', 'Nonfiction', 'History', 'Cultura...",the cathedral of notre dame de ...
8,79240,4,18,5,7232,425,5.0,043965548X,9.780440e+12,Harry Potter and the Prisoner of Azkaban,4.53,1832823.0,"['J.K. Rowling', 'Mary GrandPré']",Harry Potter's third year at Hogwarts is full ...,9.780440e+12,Harry Potter and the Prisoner of Azkaban,"['Fantasy', 'Young Adult', 'Fiction']",harry potter s third year at...
12,266347,4,24,5,7232,42,6.0,439139600,9.780439e+12,Harry Potter and the Goblet of Fire,4.53,1753043.0,"['J.K. Rowling', 'Mary GrandPré']",Harry Potter is midway through his training as...,9.780440e+12,Harry Potter and the Goblet of Fire,"['Fantasy', 'Young Adult', 'Fiction']",harry potter is midway through ...


In [79]:
recommend_using_word2vec("Barrel Fever: Stories and Essays")

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
60,960442,4,426,3,7232,13,310612.0,802130208,9.780802e+12,A Confederacy of Dunces,3.89,170776.0,"['John Kennedy Toole', 'Walker Percy']",Alternate cover for this ISBN can be found her...,9.780800e+12,A Confederacy of Dunces,"['Fiction', 'Classics', 'Humor', 'Literature']",alternate cover for this isbn ...
112,1402517,15,148,4,18205,244,2865.0,452287022,9.780452e+12,Girl with a Pearl Earring,3.85,467577.0,['Tracy Chevalier'],"With precisely 35 canvases to his credit, the ...",9.780450e+12,Girl with a Pearl Earring,"['Historical', 'Historical Fiction', 'Fiction'...",with precisely 35 canvases to ...
163,1584136,22,168,5,48089,392,149267.0,385199570,9.780385e+12,The Stand,4.34,438832.0,"['Stephen King', 'Bernie Wrightson']",This is the way the world ends: with a nanosec...,9.780390e+12,The Stand,"['Horror', 'Fiction', 'Fantasy', 'Science Fict...",this is the way the world ...
91,1201256,11,522,3,27151,106,49628.0,375507256,9.780376e+12,Cloud Atlas,4.01,152441.0,['David Mitchell'],"Cloud Atlas begins in 1850 with Adam Ewing, an...",9.780380e+12,Cloud Atlas,"['Fiction', 'Science Fiction', 'Fantasy', 'His...",cloud atlas begins in 1850 wi...
114,1442809,15,1144,4,18205,254,14249.0,60959037,9.780061e+12,Prodigal Summer,3.98,78529.0,['Barbara Kingsolver'],Prodigal Summer weaves together three stories ...,9.780060e+12,Prodigal Summer,"['Fiction', 'Contemporary', 'Environment', 'Na...",prodigal summer weaves together ...
136,1499990,18,9287,2,4559,223,33356.0,349117624,9.780349e+12,Jennifer Government,3.65,9722.0,['Max Barry'],"In Max Barry's twisted, hilarious and terrifyi...",9.780350e+12,Jennifer Government,"['Fiction', 'Science Fiction', 'Science Fictio...",in max barry s twisted ...
141,1505323,22,284,4,48089,189,70401.0,140042598,9.780140e+12,On the Road,3.64,246602.0,['Jack Kerouac'],When Jack Kerouac’s On the Road first appeared...,9.780140e+12,On the Road,"['Classics', 'Fiction', 'Travel', 'Literature']",when jack kerouac s on th...
96,1218226,11,3703,3,27151,463,11734.0,99282194,9.780099e+12,The Human Stain,3.84,21724.0,['Philip Roth'],"It is 1998, the year in which America is whipp...",9.780100e+12,The Human Stain,"['Fiction', 'Novels', 'Literature', 'American'...",it is 1998 the year in ...
61,966440,9,658,4,34632,444,3805.0,1841156736,9.781841e+12,The Corrections,3.78,117657.0,['Jonathan Franzen'],Winner of the National Book Award for FictionN...,9.781840e+12,The Corrections,"['Fiction', 'Contemporary', 'Novels']",winner of the national book a...


## Transformer (BERT)

In [65]:
def recommend_using_bert(name_of_book):
    if name_of_book not in list(books_df['book_title']):
        print('Book Chosen not available')
        return
    else:
        idx = books_df[books_df['original_title']==name_of_book].index.item()
        return recommend(bert_vectors, idx)

In [66]:
recommend_using_bert('The Da Vinci Code')

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
8,79240,4,18,5,7232,425,5.0,043965548X,9.780440e+12,Harry Potter and the Prisoner of Azkaban,4.53,1832823.0,"['J.K. Rowling', 'Mary GrandPré']",Harry Potter's third year at Hogwarts is full ...,9.780440e+12,Harry Potter and the Prisoner of Azkaban,"['Fantasy', 'Young Adult', 'Fiction']",harry potter s third year at...
6,72687,4,264,3,7232,432,3876.0,743297334,9.780743e+12,The Sun Also Rises,3.83,284988.0,['Ernest Hemingway'],"Paris in the twenties: Pernod, parties and exp...",9.780680e+12,The Sun Also Rises,"['Classics', 'Fiction', 'Literature', 'Novels']",paris in the twenties perno...
5,72564,2,8519,5,41929,369,401876.0,385337388,9.780385e+12,The House of God,3.95,8812.0,"['Samuel Shem', 'John Updike']",The hilarious novel of the healing arts that r...,9.780390e+12,The House of God,"['Health', 'Medicine', 'Fiction', 'Medical', '...",the hilarious novel of the he...
9,95095,4,27,5,7232,354,1.0,439785960,9.780440e+12,Harry Potter and the Half-Blood Prince,4.54,1678823.0,"['J.K. Rowling', 'Mary GrandPré']",When Harry Potter and the Half-Blood Prince op...,9.780440e+12,Harry Potter and the Half-Blood Prince,"['Fantasy', 'Young Adult', 'Fiction']",when harry potter and the hal...
4,49199,2,301,5,41929,194,4900.0,1892295490,9.781892e+12,Heart of Darkness,3.42,255576.0,['Joseph Conrad'],Dark allegory describes the narrator's journey...,9.781890e+12,Heart of Darkness,"['Classics', 'Fiction', 'Literature', 'Cultura...",dark allegory describes the narr...
3,15287,2,33,4,41929,98,930.0,739326228,9.780739e+12,Memoirs of a Geisha,4.08,1300209.0,['Arthur Golden'],"A literary sensation and runaway bestseller, t...",9.780740e+12,Memoirs of a Geisha,"['Fiction', 'Historical', 'Historical Fiction'...",a literary sensation and runaway...
7,76848,4,388,4,7232,491,50.0,689840926,9.780690e+12,Hatchet,3.68,222553.0,['Gary Paulsen'],Brian is on his way to Canada to visit his est...,9.780690e+12,Hatchet,"['Young Adult', 'Fiction', 'Adventure', 'Class...",brian is on his way to can...
10,110176,4,21,5,7232,12,2.0,439358078,9.780439e+12,Harry Potter and the Order of the Phoenix,4.46,1735368.0,"['J.K. Rowling', 'Mary GrandPré']",There is a door at the end of a silent corrido...,9.780440e+12,Harry Potter and the Order of the Phoenix,"['Fantasy', 'Young Adult', 'Fiction']",there is a door at the end...
2,13461,2,315,3,41929,193,4894.0,91883768,9.780092e+12,Who Moved My Cheese?,3.74,237272.0,"['Spencer Johnson', 'Kenneth H. Blanchard']","Written by Spencer Johnson, coauthor of The On...",9.780090e+12,Who Moved My Cheese?,"['Business', 'Self Help', 'Nonfiction', 'Psych...",written by spencer johnson ...


In [67]:
recommend_using_bert("Harry Potter and the Prisoner of Azkaban")

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
9,95095,4,27,5,7232,354,1.0,439785960,9.780440e+12,Harry Potter and the Half-Blood Prince,4.54,1678823.0,"['J.K. Rowling', 'Mary GrandPré']",When Harry Potter and the Half-Blood Prince op...,9.780440e+12,Harry Potter and the Half-Blood Prince,"['Fantasy', 'Young Adult', 'Fiction']",when harry potter and the hal...
10,110176,4,21,5,7232,12,2.0,439358078,9.780439e+12,Harry Potter and the Order of the Phoenix,4.46,1735368.0,"['J.K. Rowling', 'Mary GrandPré']",There is a door at the end of a silent corrido...,9.780440e+12,Harry Potter and the Order of the Phoenix,"['Fantasy', 'Young Adult', 'Fiction']",there is a door at the end...
11,125434,4,23,5,7232,369,15881.0,439064864,9.780439e+12,Harry Potter and the Chamber of Secrets,4.37,1779331.0,"['J.K. Rowling', 'Mary GrandPré']",The Dursleys were so mean and hideous that sum...,9.780440e+12,Harry Potter and the Chamber of Secrets,"['Fantasy', 'Young Adult', 'Fiction']",the dursleys were so mean and...
7,76848,4,388,4,7232,491,50.0,689840926,9.780690e+12,Hatchet,3.68,222553.0,['Gary Paulsen'],Brian is on his way to Canada to visit his est...,9.780690e+12,Hatchet,"['Young Adult', 'Fiction', 'Adventure', 'Class...",brian is on his way to can...
17,301904,4,660,3,7232,479,7672.0,60541830,9.780061e+12,Congo,3.55,129811.0,['Michael Crichton'],"Deep in the African rain forest, near the lege...",9.780060e+12,Congo,"['Fiction', 'Science Fiction', 'Thriller', 'Ad...",deep in the african rain fore...
6,72687,4,264,3,7232,432,3876.0,743297334,9.780743e+12,The Sun Also Rises,3.83,284988.0,['Ernest Hemingway'],"Paris in the twenties: Pernod, parties and exp...",9.780680e+12,The Sun Also Rises,"['Classics', 'Fiction', 'Literature', 'Novels']",paris in the twenties perno...
18,304229,4,111,2,7232,391,10441.0,143037145,9.780143e+12,The Memory Keeper's Daughter,3.64,501430.0,['Kim Edwards'],"On a winter night in 1964, Dr. David Henry is ...",9.780140e+12,The Memory Keeper's Daughter,"['Fiction', 'Contemporary', 'Adult Fiction', '...",on a winter night in 1964 ...
12,266347,4,24,5,7232,42,6.0,439139600,9.780439e+12,Harry Potter and the Goblet of Fire,4.53,1753043.0,"['J.K. Rowling', 'Mary GrandPré']",Harry Potter is midway through his training as...,9.780440e+12,Harry Potter and the Goblet of Fire,"['Fantasy', 'Young Adult', 'Fiction']",harry potter is midway through ...
5,72564,2,8519,5,41929,369,401876.0,385337388,9.780385e+12,The House of God,3.95,8812.0,"['Samuel Shem', 'John Updike']",The hilarious novel of the healing arts that r...,9.780390e+12,The House of God,"['Health', 'Medicine', 'Fiction', 'Medical', '...",the hilarious novel of the he...


In [68]:
recommend_using_bert("Harry Potter and the Prisoner of Azkaban")

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
9,95095,4,27,5,7232,354,1.0,439785960,9.780440e+12,Harry Potter and the Half-Blood Prince,4.54,1678823.0,"['J.K. Rowling', 'Mary GrandPré']",When Harry Potter and the Half-Blood Prince op...,9.780440e+12,Harry Potter and the Half-Blood Prince,"['Fantasy', 'Young Adult', 'Fiction']",when harry potter and the hal...
10,110176,4,21,5,7232,12,2.0,439358078,9.780439e+12,Harry Potter and the Order of the Phoenix,4.46,1735368.0,"['J.K. Rowling', 'Mary GrandPré']",There is a door at the end of a silent corrido...,9.780440e+12,Harry Potter and the Order of the Phoenix,"['Fantasy', 'Young Adult', 'Fiction']",there is a door at the end...
11,125434,4,23,5,7232,369,15881.0,439064864,9.780439e+12,Harry Potter and the Chamber of Secrets,4.37,1779331.0,"['J.K. Rowling', 'Mary GrandPré']",The Dursleys were so mean and hideous that sum...,9.780440e+12,Harry Potter and the Chamber of Secrets,"['Fantasy', 'Young Adult', 'Fiction']",the dursleys were so mean and...
7,76848,4,388,4,7232,491,50.0,689840926,9.780690e+12,Hatchet,3.68,222553.0,['Gary Paulsen'],Brian is on his way to Canada to visit his est...,9.780690e+12,Hatchet,"['Young Adult', 'Fiction', 'Adventure', 'Class...",brian is on his way to can...
17,301904,4,660,3,7232,479,7672.0,60541830,9.780061e+12,Congo,3.55,129811.0,['Michael Crichton'],"Deep in the African rain forest, near the lege...",9.780060e+12,Congo,"['Fiction', 'Science Fiction', 'Thriller', 'Ad...",deep in the african rain fore...
6,72687,4,264,3,7232,432,3876.0,743297334,9.780743e+12,The Sun Also Rises,3.83,284988.0,['Ernest Hemingway'],"Paris in the twenties: Pernod, parties and exp...",9.780680e+12,The Sun Also Rises,"['Classics', 'Fiction', 'Literature', 'Novels']",paris in the twenties perno...
18,304229,4,111,2,7232,391,10441.0,143037145,9.780143e+12,The Memory Keeper's Daughter,3.64,501430.0,['Kim Edwards'],"On a winter night in 1964, Dr. David Henry is ...",9.780140e+12,The Memory Keeper's Daughter,"['Fiction', 'Contemporary', 'Adult Fiction', '...",on a winter night in 1964 ...
12,266347,4,24,5,7232,42,6.0,439139600,9.780439e+12,Harry Potter and the Goblet of Fire,4.53,1753043.0,"['J.K. Rowling', 'Mary GrandPré']",Harry Potter is midway through his training as...,9.780440e+12,Harry Potter and the Goblet of Fire,"['Fantasy', 'Young Adult', 'Fiction']",harry potter is midway through ...
5,72564,2,8519,5,41929,369,401876.0,385337388,9.780385e+12,The House of God,3.95,8812.0,"['Samuel Shem', 'John Updike']",The hilarious novel of the healing arts that r...,9.780390e+12,The House of God,"['Health', 'Medicine', 'Fiction', 'Medical', '...",the hilarious novel of the he...


In [70]:
recommend_using_bert("The Hate U Give")

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
4144,6681929,2886,811,3,21768,119,18812405.0,778316556,9.780778e+12,The Good Girl,3.81,100936.0,['Mary Kubica'],I've been following her for the past few days....,9.780780e+12,The Good Girl,"['Mystery', 'Fiction', 'Thriller', 'Thriller',...",i ve been following her for ...
4280,6844223,11112,6699,5,39187,208,26154406.0,1451686633,9.781452e+12,The Passenger,3.71,13544.0,['Lisa Lutz'],"In case you were wondering, I didn’t do it. I ...",9.781450e+12,The Passenger,"['Mystery', 'Fiction', 'Thriller', 'Suspense',...",in case you were wondering ...
4005,6583155,38535,9131,4,5639,85,18161265.0,000735049X,9.780007e+12,How to Fall in Love,4.02,9486.0,['Cecelia Ahern'],She has just two weeks. Two weeks to teach him...,9.780010e+12,How to Fall in Love,"['Romance', 'Womens Fiction', 'Chick Lit', 'Fi...",she has just two weeks t...
3869,6440042,31071,8473,3,32581,88,15793231.0,1250021529,9.781250e+12,If You Find Me,4.06,13683.0,['Emily Murdoch'],\r\r\r\r\r\r\nThere are some things you can’t ...,9.781250e+12,If You Find Me,"['Young Adult', 'Contemporary', 'Fiction', 'My...",there are some things you can...
4232,6824202,27460,7148,5,35622,237,24934065.0,698410424,9.780698e+12,Rebel of the Sands,3.96,16789.0,['Alwyn Hamilton'],"Mortals rule the desert nation of Miraji, but ...",9.780700e+12,Rebel of the Sands,"['Fantasy', 'Young Adult', 'Romance', 'Fantasy...",mortals rule the desert nation ...
4132,6675633,11960,4299,3,12032,19,20443207.0,374384711,9.780374e+12,The Winner's Crime,4.17,31721.0,['Marie Rutkoski'],Book two of the dazzling Winner's Trilogy is a...,9.780370e+12,The Winner's Crime,"['Fantasy', 'Young Adult', 'Romance']",book two of the dazzling winn...
3837,6390087,24866,3980,5,13182,31,16718083.0,751552615,9.780752e+12,The Redemption of Callie & Kayden,4.11,38452.0,['Jessica Sorensen'],From the New York Times bestselling author of ...,9.780750e+12,The Redemption of Callie & Kayden,"['New Adult', 'Romance', 'Contemporary', 'Soci...",from the new york times bests...
4066,6627111,35230,3245,5,31562,167,15839984.0,62224735,9.780062e+12,Cruel Beauty,3.74,45721.0,['Rosamund Hodge'],The romance of Beauty and the Beast meets the ...,9.780060e+12,Cruel Beauty,"['Fantasy', 'Young Adult', 'Romance', 'Retelli...",the romance of beauty and the...
4283,6844829,827,9340,4,40738,77,25893582.0,553535641,9.780554e+12,Tell Me Three Things,4.10,15565.0,['Julie Buxbaum'],"Everything about Jessie is wrong. At least, th...",9.780550e+12,Tell Me Three Things,"['Young Adult', 'Contemporary', 'Romance', 'Fi...",everything about jessie is wrong...


In [71]:
recommend_using_bert("How to Fall in Love")

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
4326,6863345,53098,9569,5,35429,250,32075671.0,62498533,9.780062e+12,The Hate U Give,4.62,32610.0,"['Angie Thomas', 'Angie Thomas']",Sixteen-year-old Starr Carter moves between tw...,9.780060e+12,The Hate U Give,"['Young Adult', 'Contemporary', 'Fiction', 'Re...",sixteen year old starr carter moves...
4144,6681929,2886,811,3,21768,119,18812405.0,778316556,9.780778e+12,The Good Girl,3.81,100936.0,['Mary Kubica'],I've been following her for the past few days....,9.780780e+12,The Good Girl,"['Mystery', 'Fiction', 'Thriller', 'Thriller',...",i ve been following her for ...
4280,6844223,11112,6699,5,39187,208,26154406.0,1451686633,9.781452e+12,The Passenger,3.71,13544.0,['Lisa Lutz'],"In case you were wondering, I didn’t do it. I ...",9.781450e+12,The Passenger,"['Mystery', 'Fiction', 'Thriller', 'Suspense',...",in case you were wondering ...
3869,6440042,31071,8473,3,32581,88,15793231.0,1250021529,9.781250e+12,If You Find Me,4.06,13683.0,['Emily Murdoch'],\r\r\r\r\r\r\nThere are some things you can’t ...,9.781250e+12,If You Find Me,"['Young Adult', 'Contemporary', 'Fiction', 'My...",there are some things you can...
4037,6599628,12312,5279,5,21186,437,18336965.0,NaN,NaN,My Heart and Other Black Holes,4.02,22311.0,['Jasmine Warga'],"Aysel and Roman are practically strangers, but...",9.781440e+12,My Heart and Other Black Holes,"['Young Adult', 'Contemporary', 'Romance', 'He...",aysel and roman are practically ...
4325,6863235,12503,8685,4,3215,37,28965131.0,125011117X,9.781250e+12,Behind Her Eyes,3.77,17944.0,['Sarah Pinborough'],Why is everyone talking about the ending of Sa...,9.781250e+12,Behind Her Eyes,"['Thriller', 'Mystery', 'Fiction', 'Suspense',...",why is everyone talking about ...
4209,6805190,15090,3345,4,44638,145,19547856.0,62348671,9.780062e+12,Simon vs. the Homo Sapiens Agenda,4.26,53570.0,['Becky Albertalli'],Sixteen-year-old and not-so-openly gay Simon S...,9.780060e+12,Simon vs. the Homo Sapiens Agenda,"['Young Adult', 'Contemporary', 'Lgbt', 'Roman...",sixteen year old and not so openly ...
3795,6324809,28116,6343,5,13659,51,16055475.0,1455574880,9.781456e+12,The Forever of Ella and Micha,4.10,24776.0,['Jessica Sorensen'],Ella and Micha have survived tragedy and heart...,9.781460e+12,The Forever of Ella and Micha,"['New Adult', 'Romance', 'Contemporary', 'Roma...",ella and micha have survived ...
3837,6390087,24866,3980,5,13182,31,16718083.0,751552615,9.780752e+12,The Redemption of Callie & Kayden,4.11,38452.0,['Jessica Sorensen'],From the New York Times bestselling author of ...,9.780750e+12,The Redemption of Callie & Kayden,"['New Adult', 'Romance', 'Contemporary', 'Soci...",from the new york times bests...


In [77]:
recommend_using_bert("Barrel Fever: Stories and Essays")

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
83,1066347,9,227,2,34632,333,5359.0,385339089,9.780385e+12,The Client,3.97,320083.0,['John Grisham'],"In a weedy lot on the outskirts of Memphis, tw...",9.780390e+12,The Client,"['Fiction', 'Mystery', 'Thriller', 'Mystery', ...",in a weedy lot on the outs...
68,998901,9,1642,4,34632,309,11334.0,140003342X,9.781400e+12,Song of Solomon,4.01,59756.0,['Toni Morrison'],Milkman Dead was born shortly after a neighbor...,9.781400e+12,Song of Solomon,"['Fiction', 'Classics', 'Cultural', 'African A...",milkman dead was born shortly ...
87,1079693,9,1311,2,34632,326,5191.0,316601950,9.780317e+12,The Pilot's Wife,3.50,71777.0,['Anita Shreve'],A pilot's wife is taught to be prepared for th...,9.780320e+12,The Pilot's Wife,"['Fiction', 'Womens Fiction', 'Chick Lit', 'My...",a pilot s wife is taught ...
93,1208257,11,516,4,27151,133,3985.0,312282990,9.780312e+12,The Amazing Adventures of Kavalier & Clay,4.17,147717.0,['Michael Chabon'],"Joe Kavalier, a young Jewish artist who has al...",9.780310e+12,The Amazing Adventures of Kavalier & Clay,"['Fiction', 'Historical', 'Historical Fiction'...",joe kavalier a young jewish...
91,1201256,11,522,3,27151,106,49628.0,375507256,9.780376e+12,Cloud Atlas,4.01,152441.0,['David Mitchell'],"Cloud Atlas begins in 1850 with Adam Ewing, an...",9.780380e+12,Cloud Atlas,"['Fiction', 'Science Fiction', 'Fantasy', 'His...",cloud atlas begins in 1850 wi...
84,1070607,9,587,4,34632,237,256566.0,60529709,9.780061e+12,Everything Is Illuminated,3.90,131492.0,['Jonathan Safran Foer'],"A young man arrives in the Ukraine, clutching ...",9.780060e+12,Everything Is Illuminated,"['Fiction', 'Historical', 'Historical Fiction'...",a young man arrives in the ...
74,1034192,9,795,4,34632,306,4271.0,140285679,9.780140e+12,About a Boy,3.79,101999.0,['Nick Hornby'],'How cool was Will Freeman?'Too cool! At thirt...,9.780140e+12,About a Boy,"['Fiction', 'Contemporary', 'Humor']",how cool was will freeman ...
86,1076549,9,407,5,34632,44,5826.0,60838728,9.780061e+12,Bel Canto,3.92,192330.0,['Ann Patchett'],"In an unnamed South American country, a world-...",9.780060e+12,Bel Canto,"['Fiction', 'Contemporary', 'Literary Fiction'...",in an unnamed south american ...
71,1015922,9,494,2,34632,114,29044.0,1400031702,9.781400e+12,The Secret History,4.07,168405.0,['Donna Tartt'],Under the influence of their charismatic class...,9.781400e+12,The Secret History,"['Fiction', 'Mystery', 'Contemporary']",under the influence of their ...


In [ ]:
#Ideas: shuffle the data before training the corpus

In [ ]:
#Use genres for validation or Expectation Maximization algorithm to cluster users

In [ ]:
#Use authors, title, genres for prediction besides book_desc